# Per-Project Summary Statistics

In [1]:
!pwd

/home/michael/dev/data-portal-summary-stats


In [2]:
import os
homepath = os.path.join(os.path.expanduser("~"), 'dev/hca-summary-stats')
os.chdir(homepath)
import scanpy as sc
import numpy as np
from src.matrix_summary_stats import MatrixSummaryStats

In [3]:
!pwd

/home/michael/dev/hca-summary-stats


In [4]:
sc.settings.set_figure_params(dpi=200)

In [5]:
matrix_service_endpoint = 'https://matrix.staging.data.humancellatlas.org/v1'
# Parameter for matrix request
feature = 'gene'
format_ = 'mtx'
project = 'Single cell transcriptome analysis of human pancreas'  # get single project
project_field = 'project.project_core.project_short_name'
min_cell_count = 300
min_cell_count_field = 'genes_detected'
list_projects_url = matrix_service_endpoint + '/filters/' + project_field
payload = {
    'feature': feature,
    'format': format_,
    'filter': {
        'op': 'and',
        'value': [
            {
                'op': '=',
                'value': project,
                'field': project_field
            },
            {
                'op': '>=',
                'value': min_cell_count,
                'field': min_cell_count_field
            }
        ]
    }
}

In [11]:
import requests
import time

In [13]:
response = requests.post(matrix_service_endpoint + '/matrix', json=payload)

In [25]:
requests.get(matrix_service_endpoint + '/projects').json()

{'message': 'Missing Authentication Token'}

In [ ]:
while True:
    status_response = requests.get(matrix_service_endpoint + '/matrix/' +
                                   response.json()['request_id'])
    if status_response.json()['status'] == 'Complete':
        break
    print(f'{status_response.json()["status"]} ...')
    time.sleep(30)

In [19]:
status_response = requests.get(matrix_service_endpoint + '/matrix/' +
                                   response.json()['request_id'])

In [21]:
status_response.json()['matrix_url']

'https://s3.amazonaws.com/dcp-matrix-service-results-staging/02f43a31-d4b4-404a-93ef-3120c20bb542.mtx.zip'

In [27]:
# Azul's prod endpoint for projects
url = "https://service.explore.data.humancellatlas.org/repository/projects/"
requests.get(url).json()

{'hits': [{'protocols': [{'libraryConstructionApproach': ['10X v2 sequencing'],
     'instrumentManufacturerModel': ['Illumina HiSeq 4000'],
     'pairedEnd': [False],
     'workflow': [],
     'assayType': []}],
   'entryId': '74b6d569-3b11-42ef-b6b1-a0454522b4a0',
   'projects': [{'projectTitle': '1.3 Million Brain Cells from E18 Mice',
     'projectShortname': '1M Neurons',
     'laboratory': ['Human Cell Atlas Data Coordination Platform'],
     'arrayExpressAccessions': [],
     'geoSeriesAccessions': ['GSE93421'],
     'insdcProjectAccessions': ['SRP096558'],
     'insdcStudyAccessions': ['PRJNA360949']}],
   'samples': [{'sampleEntityType': ['specimens'],
     'effectiveOrgan': ['brain'],
     'organPart': ['cortex'],
     'source': ['specimen_from_organism'],
     'organ': ['brain'],
     'id': ['E18_20161004_Brain', 'E18_20160930_Brain'],
     'preservationMethod': ['fresh'],
     'disease': ['normal']}],
   'specimens': [{'id': ['E18_20161004_Brain', 'E18_20160930_Brain'],
   

In [ ]:
#matrix_path = 'test/data/04b7e4ff-a4fd-4a77-9453-16cd0f7ca030.mtx.zip'
matrix_path = 'test/data/5ff7733b-916f-4cca-b466-99b1df250e25.mtx.zip'
mss = MatrixSummaryStats()

In [ ]:
mss.matrix_zipfile_name

In [ ]:
mss.unzip_files(path='test/data')

In [ ]:
adata = sc.read_10x_mtx(
    './test/data/04b7e4ff-a4fd-4a77-9453-16cd0f7ca030.mtx',      # the directory with the `.mtx` file
    var_names='gene_symbols',   # use gene symbols for the variable names (variables-axis index)
    cache=True)
adata.var_names_make_unique()

In [ ]:
adata

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=10)

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
mito_genes = adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

In [ ]:
adata.var_keys()
#group_key = adata.obs['cell_suspension.provenance.document_id']
#sc.pl.stacked_violin(adata, list(adata.var_names)[:5], swap_axis=True)
#sc.pl.stacked_violin(adata[:10], expression_variance, grouby=group_key, swap_axis=True, use_raw=False)

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
adata

In [ ]:
adata = adata[adata.obs['n_genes'] < 2500, :]
adata = adata[adata.obs['percent_mito'] < 0.05, :]

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)  # logarithmize
adata.raw = adata   # save raw data for later use

### Identify highly variable genes.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
adata = sc.datasets.pbmc68k_reduced()  # scanpy demo data
sc.pl.highly_variable_genes(adata)